# Chocolate Milkshake Data -- small subset

In [ ]:
#this imports all the commands needed for the script to work#
import os
import numpy as np
import nilearn
import glob
#import matplotlib
import nibabel as nib
import pandas as pd 
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')

In [ ]:
#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/power_roimask_4bi.nii.gz'
#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/w1_milkshake_48.csv'
#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/w1_milkshake_48.nii.gz'
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")
#grab conditional labels 
conditions = behavioral["Label"]

#confirm we have the # of condtions needed
print(conditions.unique())



In [5]: print(conditions.unique())  
['rest' 'milkshake_pic' 'LF_HS_receipt' 'rinse' 'HF_HS_receipt'  
 'LF_LS_receipt' 'h20_pic' 'h20_receipt' 'HF_LS_receipt']  

### LF_HS_receipt vs. h20_receipt

In [ ]:
#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['LF_HS_receipt', "h20_receipt"])
conditions = conditions[condition_mask]

#confirm we have the # of condtions needed
print(conditions.unique())

# Record these as an array of sessions, with fields
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

In [ ]:
# For decoding, standardizing is often very important
# note that we are also smoothing the data
masker = NiftiMasker(mask_img=imag_mask,
                     standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')

from sklearn.feature_selection import SelectKBest, f_classif
feature_selection = SelectKBest(f_classif, k=500)

# We have our classifier (SVC), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

In [ ]:
anova_svc.fit(X, conditions)
y_pred = anova_svc.predict(X)

In [ ]:
from sklearn.model_selection import cross_val_score

k_range = [10, 15, 30, 50, 150, 300, 500, 1000, 1500, 3000, 5000]

from sklearn.model_selection import GridSearchCV
# We are going to tune the parameter 'k' of the step called 'anova' in
# the pipeline. Thus we need to address it as 'anova__k'.

# Note that GridSearchCV takes an n_jobs argument that can make it go
# much faster
grid = GridSearchCV(anova_svc, param_grid={'anova__k': k_range}, verbose=1, n_jobs=4, cv=3)
nested_cv_scores = cross_val_score(grid, X, conditions, cv=3)

print("Nested CV score: %.4f" % np.mean(nested_cv_scores))

In [ ]:
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)


# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(dataset)

""" display.savefig('pretty_brain.png')"""
# Create the figure
from nilearn.plotting import plot_stat_map, show
display = plot_stat_map(weight_img, mean_img, title='SVM weights')
display.savefig('svm_milkshake_48_HF_LS_h20.png')
# Saving the results as a Nifti file may also be important
weight_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/milkshake_svm_48_HF_LS_h20.nii')

Fitting 3 folds for each of 11 candidates, totalling 33 fits  
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.  
[Parallel(n_jobs=4)]: Done  33 out of  33 | elapsed: 30.2min finished  
Fitting 3 folds for each of 11 candidates, totalling 33 fits  
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.    
[Parallel(n_jobs=4)]: Done  33 out of  33 | elapsed: 29.8min finished  
Fitting 3 folds for each of 11 candidates, totalling 33 fits  
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.  
[Parallel(n_jobs=4)]: Done  33 out of  33 | elapsed: 30.1min finished  
Nested CV score: 0.5035  

### LF_LS_receipt vs. h20_receipt

In [ ]:
#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/power_roimask_4bi.nii.gz'
#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/w1_milkshake_100.csv'
#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/w1_milkshake_100.nii.gz'
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")
#grab conditional labels 
conditions = behavioral["Label"]
#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['LF_LS_receipt', "h20_receipt"])
conditions = conditions[condition_mask]
#confirm we have the # of condtions needed
print(conditions.unique())

# Record these as an array of sessions, with fields
# for condition (face or house) and run
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

In [ ]:
# For decoding, standardizing is often very important
# note that we are also smoothing the data
masker = NiftiMasker(mask_img=imag_mask,
                     standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')

from sklearn.feature_selection import SelectKBest, f_classif
feature_selection = SelectKBest(f_classif, k=500)

# We have our classifier (SVC), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

In [ ]:
anova_svc.fit(X, conditions)
y_pred = anova_svc.predict(X)

In [ ]:
from sklearn.model_selection import cross_val_score

k_range = [10, 15, 30, 50, 150, 300, 500, 1000, 1500, 3000, 5000]

from sklearn.model_selection import GridSearchCV
# We are going to tune the parameter 'k' of the step called 'anova' in
# the pipeline. Thus we need to address it as 'anova__k'.

# Note that GridSearchCV takes an n_jobs argument that can make it go
# much faster
grid = GridSearchCV(anova_svc, param_grid={'anova__k': k_range}, verbose=1, n_jobs=4, cv=3)
nested_cv_scores = cross_val_score(grid, X, conditions, cv=3)

print("Nested CV score: %.4f" % np.mean(nested_cv_scores))

Fitting 3 folds for each of 11 candidates, totalling 33 fits
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.

[Parallel(n_jobs=4)]: Done  33 out of  33 | elapsed: 29.6min finished
Fitting 3 folds for each of 11 candidates, totalling 33 fits
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.



[Parallel(n_jobs=4)]: Done  33 out of  33 | elapsed: 29.5min finished
Fitting 3 folds for each of 11 candidates, totalling 33 fits
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.


[Parallel(n_jobs=4)]: Done  33 out of  33 | elapsed: 29.7min finished
Nested CV score: 0.4985



In [ ]:
behavioral = pd.read_csv(stim, sep="\t")
#grab conditional labels 
conditions = behavioral["Label"]

#confirm we have the # of condtions needed
print(conditions.unique())


### HF_LS vs. h20_receipt 

In [ ]:
behavioral = pd.read_csv(stim, sep="\t")
#grab conditional labels 
conditions = behavioral["Label"]

#confirm we have the # of condtions needed
print(conditions.unique())

#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['HF_LS_receipt', "h20_receipt"])
conditions = conditions[condition_mask]

#confirm we have the # of condtions needed
print(conditions.unique())

# Record these as an array of sessions, with fields
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

# For decoding, standardizing is often very important
# note that we are also smoothing the data
masker = NiftiMasker(mask_img=imag_mask,
                     standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]


In [ ]:
behavioral = pd.read_csv(stim, sep="\t")
#grab conditional labels 
conditions = behavioral["Label"]

#confirm we have the # of condtions needed
print(conditions.unique())

#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin([''', "h20_receipt"])
conditions = conditions[condition_mask]

#confirm we have the # of condtions needed
print(conditions.unique())

# Record these as an array of sessions, with fields
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

# For decoding, standardizing is often very important
# note that we are also smoothing the data
masker = NiftiMasker(mask_img=imag_mask,
                     standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
# Apply our condition_mask
X = X[condition_mask]
